In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import zipfile
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import torchvision

# Define your model
class IRShuffleUnit(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(IRShuffleUnit, self).__init__()
        self.stride = stride
        mid_channels = in_channels * 2
        self.pwconv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.dwconv = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=1, groups=mid_channels, bias=False)
        self.pwconv2 = nn.Conv2d(mid_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.bn2 = nn.BatchNorm2d(mid_channels)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.prelu = nn.PReLU()

    def forward(self, x):
        out = self.pwconv1(x)
        out = self.bn1(out)
        out = self.prelu(out)
        out = self.dwconv(out)
        out = self.bn2(out)
        out = self.prelu(out)
        out = self.pwconv2(out)
        out = self.bn3(out)
        return out

class LightweightCNN(nn.Module):
    def __init__(self, num_classes):
        super(LightweightCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.irshuffle_blocks = nn.Sequential(
            IRShuffleUnit(32, 48, 2),
            IRShuffleUnit(48, 48, 1),
            IRShuffleUnit(48, 96, 2),
            IRShuffleUnit(96, 96, 1),
            IRShuffleUnit(96, 96, 1),
            IRShuffleUnit(96, 96, 1),
            IRShuffleUnit(96, 192, 2),
            IRShuffleUnit(192, 192, 1)
        )
        self.pwconv1 = nn.Conv2d(192, 512, kernel_size=1, bias=False)
        self.dwconv = nn.Conv2d(512, 512, kernel_size=3, padding=1, groups=512, bias=False)
        self.pwconv2 = nn.Conv2d(512, 128, kernel_size=1, bias=False)
        self.fc = nn.Linear(128 * 7 * 6, num_classes)  # Modify the input size accordingly

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.irshuffle_blocks(x)
        x = F.relu(self.pwconv1(x))
        x = F.relu(self.dwconv(x))
        x = self.pwconv2(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)  # Apply the fully connected layer
        return x

# Create the dataset and dataloader
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.img_paths = []
        self.labels = []
        self.class_to_idx = {}

        for root, _, files in os.walk(root_dir):
            class_name = os.path.basename(root)
            if class_name not in self.class_to_idx:
                self.class_to_idx[class_name] = len(self.class_to_idx)
            for file in files:
                if file.endswith(('png', 'jpg', 'jpeg')):
                    self.img_paths.append(os.path.join(root, file))
                    self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        label = self.labels[idx]
        return img, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((112, 96)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Upload and extract dataset
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]
extracted_path = '/content/dataset'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

dataset = ImageDataset(extracted_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

# Instantiate the model with the correct number of classes
model = LightweightCNN(num_classes=len(dataset.class_to_idx))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Saving archive (2).zip to archive (2).zip


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import torch.nn.functional as F

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)

        if labels.dim() != 1:
            labels = labels.view(-1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')

Epoch 0/24, Loss: 8.1733
Epoch 1/24, Loss: 7.3125
Epoch 2/24, Loss: 6.1389
Epoch 3/24, Loss: 4.5568
Epoch 4/24, Loss: 2.9557
Epoch 5/24, Loss: 1.8846
Epoch 6/24, Loss: 1.2749
Epoch 7/24, Loss: 0.8776
Epoch 8/24, Loss: 0.6535
Epoch 9/24, Loss: 0.5124
Epoch 10/24, Loss: 0.4158
Epoch 11/24, Loss: 0.3754
Epoch 12/24, Loss: 0.3078
Epoch 13/24, Loss: 0.3012
Epoch 14/24, Loss: 0.2585
Epoch 15/24, Loss: 0.2168
Epoch 16/24, Loss: 0.2312
Epoch 17/24, Loss: 0.2411
Epoch 18/24, Loss: 0.1954
Epoch 19/24, Loss: 0.2041
Epoch 20/24, Loss: 0.1905
Epoch 21/24, Loss: 0.1718
Epoch 22/24, Loss: 0.1597
Epoch 23/24, Loss: 0.1524
Epoch 24/24, Loss: 0.1548


In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'lightweight_cnn.pth')

In [ ]:
# Function to evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    total_time = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            start_time = time.time()
            outputs = model(inputs)
            end_time = time.time()

            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            total_time += end_time - start_time

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    avg_inference_time = total_time / len(all_labels)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'Average Inference Time per Image: {avg_inference_time:.6f} seconds')

# Evaluate the model
evaluate_model(model, dataloader)

Accuracy: 0.9636
Precision: 0.9684
Recall: 0.9636
F1 Score: 0.9630
Average Inference Time per Image: 0.012824 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, log_loss, matthews_corrcoef, cohen_kappa_score, classification_report
import seaborn as sns

In [ ]:
# Function to calculate log loss
def calculate_log_loss(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            preds = torch.softmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    loss = log_loss(all_labels, all_preds)
    print(f'Log Loss: {loss:.4f}')

In [ ]:
# Load the model
num_classes = len(dataset.class_to_idx)  # Ensure this matches your dataset
model = LightweightCNN(num_classes)
model.load_state_dict(torch.load('lightweight_cnn.pth'))
model.eval()

LightweightCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (irshuffle_blocks): Sequential(
    (0): IRShuffleUnit(
      (pwconv1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (dwconv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (pwconv2): Conv2d(64, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
    )
    (1): IRShuffleUnit(
      (pwconv1): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (dwconv): Conv2d(96, 96, kernel_size=(3, 3), stride=(

In [ ]:
# Function to measure latency
def measure_latency(model, image_path, transform, num_iterations=100):
    img = Image.open(image_path).convert('RGB')
    img = transform(img).unsqueeze(0)  # Add batch dimension

    # Warm up the model
    for _ in range(10):
        _ = model(img)

    # Measure latency
    start_time = time.time()
    for _ in range(num_iterations):
        _ = model(img)
    end_time = time.time()

    avg_latency = (end_time - start_time) / num_iterations
    return avg_latency

# Upload a file
#uploaded = files.upload()

# Check if the uploaded file is an image or a zip file
uploaded_file = list(uploaded.keys())[0]
if uploaded_file.endswith(('.jpg', '.jpeg', '.png')):
    image_path = uploaded_file
elif uploaded_file.endswith('.zip'):
    with zipfile.ZipFile(uploaded_file, 'r') as zip_ref:
        zip_ref.extractall('/content/')
        # Find the first image file in the extracted content
        for root, _, files in os.walk('/content/'):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(root, file)
                    break

# Measure latency for the image
avg_latency = measure_latency(model, image_path, transform)
print(f'Average latency per image: {avg_latency:.6f} seconds')

Average latency per image: 0.026631 seconds


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

def plot_roc_curve(model, dataloader, num_classes):
    model.eval()
    all_labels = []
    all_probs = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    all_labels = label_binarize(all_labels, classes=range(num_classes))
    fpr = {}
    tpr = {}
    roc_auc = {}

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(all_labels[:, i], [prob[i] for prob in all_probs])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(10, 8))
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

# Plot ROC curve for validation set
plot_roc_curve(model, val_dataloader, len(dataset.class_to_idx))

NameError: name 'val_dataloader' is not defined